In [ ]:
!pip install sentence_transformers
!pip install urllib3
!pip install language-tool-python 

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import urllib.request
from sentence_transformers import SentenceTransformer
import re
import time
from tqdm import tqdm
import os
import language_tool_python
from multiprocessing import Process

# SBERT
## Similarity_Checker

In [ ]:
df_right_answers = pd.read_csv('/content/drive/MyDrive/final project/Dataset/input_data/right_ans/EN_right_ans.csv',index_col=0)
df_right_answers

In [ ]:
wai_path = '/content/drive/MyDrive/final project/Dataset/input_data'  # for_valid 폴더 생성  
wai_questions = os.listdir(wai_path)
wai_questions = [p for p in wai_questions if 'csv' in p]
wai_questions  # 컬럼명 질문 내용 답변인 원본 데이터 프레임

### 테스트용 문장

In [ ]:
sentences = ['You are so beautiful', 'You are so kind', 'You are too bad', 'You are too noisy']

### Sentence Embedding
- 다양한 사전 학습 모델을 통해 문장 embedding 결과\
 "xlm-r-large-en-ko-nli-ststb" 가 가장 적합하다 판단

In [ ]:
model_ml_L6 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings_L6 = model_ml_L6.encode(sentences)
sentences, embeddings_L6

In [ ]:
model_ml_all = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings_all = model_ml_all.encode(sentences)
sentences, embeddings_all

In [ ]:
model_ml = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
embeddings = model_ml.encode(sentences)
sentences, embeddings

In [ ]:
sbert_EN = SentenceTransformer('sentence-transformers/xlm-r-large-en-ko-nli-ststb')
embeddings_EN = sbert_EN.encode(sentences)
embeddings_EN

## 유사도 구하는 함수

In [ ]:
pairs = []
for i, emb in enumerate(embeddings_EN):
    for j in range(i+1, len(embeddings_EN)):
        pairs.append((sentences[i], sentences[j],emb, embeddings_EN[j]))

In [ ]:
def get_similarity(ans, right_ans, use="cosine"):
    # Cosine Similarity
    if use == "cosine":
        return dot(ans, right_ans)/(norm(ans)*norm(right_ans))
        
    # # Euclidean
    # if use == "euclidean":
    #     if norm(ans-right_ans)==norm(ans-right_ans):
    #         return norm(ans-right_ans)
    #     else:
    #         return -1
    
    
    # # Pearson
    # if use == "pearson":
    #     return dot((ans - np.mean(ans)), (right_ans - np.mean(right_ans))) / ((norm(ans - np.mean(ans))) * (norm(right_ans - np.mean(right_ans))))

In [ ]:
for (sent1,sent2, ans, r_ans) in pairs:
    print(sent1," // ", sent2, get_similarity(ans, r_ans))

## Process

In [ ]:
pass_list = ['cosine_similarity','grammar_checker','keyword_sum','total_score']

for path in pass_list:
    os.makedirs("/content/drive/MyDrive/final project/Dataset/output_data/"+path,  exist_ok=True)

In [ ]:
def sbert_final(pathname):
  for q in wai_questions:
      q_pd = pd.read_csv(os.path.join(pathname,q),index_col=0)
      
      start = time.time()  # 시작 시간 저장
      
      # # 전처리
      # question = re.sub(r"[^a-zA-Z가-힣\s\t\?\']", "", q_pd.columns[0])
      # print(question)
      
      # answers = list(str(q_pd[q_pd.columns[0]]))
      answers=[]
      for i in range(len(q_pd[q_pd.columns[0]])):
        a=str(q_pd[q_pd.columns[0]][i])
        answers.append(a)
      # answers = [re.sub(r"[^a-zA-Z가-힣\s\t\.\']", "", str(ans)) for ans in answers if ans==ans]
      # answers = [ans for ans in answers if ans!=""]
      
      # print(f"전처리 시간 : {time.time() - start:.3f} 초")  # 현재시각 - 시작시간 = 실행 시간
      # print()
      # start = time.time()

      # new_preprocessing
      # answer = []
      # for ans in answers:
      #   if ans == ans:
      #     a = re.sub(r"[^a-zA-Z가-힣\s\t\.\']", "", str(ans))
      #     answer.append(a)
      
      # 한 답변씩 유사도 검사
      print(q.split('_')[2].split('.')[0])
      print(df_right_answers['question_num']=="q"+q.split('_')[2].split('.')[0])

      right_answers = list(df_right_answers[df_right_answers['question_num']=="q"+q.split('_')[2].split('.')[0]]['right_ans'])
      right_answers_emb = sbert_EN.encode(right_answers)
      
      print(f"모범답안 embedding 시간 : {time.time() - start:.3f} 초")  # 현재시각 - 시작시간 = 실행 시간
      print()
      start = time.time()
      
      
      similarity_cos = []
      similarity_euclidean = []
      similarity_pearson = []
      similarity_total = []
      for i in tqdm(range(len(answers))):
          ans = answers[i]
          ans_emb = sbert_EN.encode([ans])
          for j in range(len(right_answers_emb)):
              r_emb = right_answers_emb[j]
              cos_sim = get_similarity(ans_emb[0], r_emb, use='cosine')
              # euclidean_sim = get_similarity(ans_emb[0], r_emb, use='euclidean')
              # pearson_sim = get_similarity(ans_emb[0], r_emb, use='pearson')
              
              similarity_cos.append((right_answers[j], ans, cos_sim))
              # similarity_euclidean.append((right_answers[j], ans, euclidean_sim))
              # similarity_pearson.append((right_answers[j], ans, pearson_sim))
              # similarity_total.append((right_answers[j], ans, cos_sim, euclidean_sim, pearson_sim))
      print(ans)
      # print(f"학생 답안 embedding & 유사도 검사 시간 : {time.time() - start:.3f} 초")  # 현재시각 - 시작시간 = 실행 시간
      print()
      start = time.time()
      
      
      # similarity 기준 sort
      similarity_cos_sorted = sorted(similarity_cos, key = lambda x : x[2], reverse=True)
      # similarity_euclidean_sorted = sorted(similarity_euclidean, key = lambda x : x[2])
      # similarity_pearson_sorted = sorted(similarity_pearson, key = lambda x : x[2], reverse=True)
      # similarity_total_sorted = sorted(similarity_total, key = lambda x : x[2], reverse=True)

      print(similarity_cos_sorted)

      # csv 저장
      df_similarity_cos = pd.DataFrame({'right_ans': [x[0] for x in similarity_cos_sorted],
                              'student_ans':[x[1] for x in similarity_cos_sorted],
                              'similarity':[x[2] for x in similarity_cos_sorted]})
      
      df_similarity_cos.to_csv(f"/content/drive/MyDrive/final project/Dataset/output_data/cosine_similarity/similarity_cos_q{q.split('_')[2].split('.')[0]}.csv")

      # df_similarity_euclidean = pd.DataFrame({'right_ans': [x[0] for x in similarity_euclidean_sorted],
      #                         'student_ans':[x[1] for x in similarity_euclidean_sorted],
      #                         'similarity':[x[2] for x in similarity_euclidean_sorted]})
      # df_similarity_euclidean.to_csv(f"/content/drive/MyDrive/final project/Dataset/euclidian_similarity/{q.split('.')[0]}_pair_with_similarity_eu.csv")

      # df_similarity_pearson = pd.DataFrame({'right_ans': [x[0] for x in similarity_pearson_sorted],
      #                         'student_ans':[x[1] for x in similarity_pearson_sorted],
      #                         'similarity':[x[2] for x in similarity_pearson_sorted]})
      # df_similarity_pearson.to_csv(f"/content/drive/MyDrive/final project/Dataset/pearson_similarity/{q.split('.')[0]}_pair_with_similarity_pearson.csv")

      # df_similarity_total = pd.DataFrame({'right_ans': [x[0] for x in similarity_total_sorted],
      #                         'student_ans':[x[1] for x in similarity_total_sorted],
      #                         'cos_similarity':[x[2] for x in similarity_total_sorted],
      #                         'euclidean_similarity':[x[3] for x in similarity_total_sorted],
      #                         'pearson_similarity':[x[4] for x in similarity_total_sorted]})
      # df_similarity_total.to_csv(f"/content/drive/MyDrive/final project/Dataset/whole_similarity/{q.split('.')[0]}_pair_with_similarity_total.csv")
      print()

In [ ]:
sbert_final(wai_path)

---

# Grammer Checker

In [ ]:
cos_path = '/content/drive/MyDrive/final project/Dataset/output_data/cosine_similarity/'   
cos_file = os.listdir(cos_path)
cos_file = [p for p in cos_file if 'csv' in p]
print(len(cos_file))  # 컬럼명 질문 내용 답변인 원본 데이터 프레임
print(cos_file)

In [ ]:
def grammer_check(filepath):

  tool = language_tool_python.LanguageTool('en-US')
  for q in filepath:
        q_pd = pd.read_csv(os.path.join(cos_path,q),index_col=0)
        # q_pd = q_pd.drop(columns = ['right_ans', 'similarity'])
        q_pd['check'] = 0
        print(q_pd)
        for i in range(len(q_pd)):
          text = q_pd.iloc[i][1]
          print(text)

          # get the matches
          matches = tool.check(text)

          print(matches)
          print(len(matches))
          q_pd['check'][i]=len(matches)
          q_pd.to_csv(f"/content/drive/MyDrive/final project/Dataset/output_data/grammar_checker/grammer_checker_{q.split('_')[2].split('.')[0]}.csv")

In [ ]:
# def grammer_check(filepath):

#   tool = language_tool_python.LanguageTool('en-US')
#   for q in filepath:
#         q_pd = pd.read_csv(os.path.join(cos_path,q),index_col=0)
#         # q_pd = q_pd.drop(columns = ['right_ans', 'similarity'])
#         q_pd['check'] = 0
#         print(q_pd)
#         for i in range(len(q_pd)):
#           text = q_pd.iloc[i][1]
#           print(text)

#           # get the matches
#           matches = tool.check(text)

#           print(matches)
#           print(len(matches))

#           my_mistakes = []
#           my_corrections = []
#           start_positions = []
#           end_positions = []

#           for rules in matches:
#             if len(rules.replacements) > 0:
#               start_positions.append(rules.offset)
#               end_positions.append(rules.errorLength + rules.offset)
#               my_mistakes.append(text[rules.offset: rules.errorLength + rules.offset])
#               my_corrections.append(rules.replacements[0])
              
#               q_pd['check'][i]=len(list(zip(my_mistakes)))
#           # my_new_text = list(text)

#           # for m in range(len(start_positions)):
#           #   for i in range(len(text)):
#           #     print(my_new_text,"-----------")
#           #     print(my_corrections[m])
#           #     my_new_text[start_positions[m]] = my_corrections[m]
#           #     if (i > start_positions[m] and i < end_positions[m]):
#           #       my_new_text[i] = ""
#           print(list(zip(my_mistakes, my_corrections)))  
#           print(len(list(zip(my_mistakes))))
#           q_pd.to_csv(f"/content/drive/MyDrive/final project/Dataset/output_data/grammar_checker/grammer_checker_{q.split('_')[2].split('.')[0]}.csv")

In [ ]:
grammer_check(cos_file)

# Keyword Checker

In [ ]:
# wai_path = '/content/drive/MyDrive/final project/Dataset/input_data'  # for_valid 폴더 생성  
# wai_questions = os.listdir(wai_path)
# wai_questions = [p for p in wai_questions if 'csv' in p]
# wai_questions  # 컬럼명 질문 내용 답변인 원본 데이터 프레임

In [ ]:
for q in wai_questions:
    q_pd = pd.read_csv(os.path.join(wai_path,q),index_col=0)
    print(q_pd.head(3))

    # 컬럼 명 변경하기(문제 번화 확인을 위해 키워드 컬럼명은 나중에 변경)
    col_list = list(q_pd.columns)    # 컬럼명 리스트로 변경 후 내용 변경
    col_list[0] = 'student_ans'
    q_pd.columns = col_list
    print(q_pd)

    # 문항별 키워드 갯수 확인
    keyword=q_pd.columns[1:]
    n = len(keyword)
    print(n)
    
    # 문항번호와 키워드 분리
    keyword_split = []
    for i in range(0,n):
      split_key= keyword[i].split('_')[1]
      keyword_split.append(split_key)
    print(keyword_split)

   # 컬럼 명 변경하기
    col_list = list(q_pd.columns)    
    col_list[1:] = keyword_split          # 분리한 키워드 활용하여 컬럼명 변경/키워드와 동일하게 변경해야 label입력 가능
    q_pd.columns = col_list
    print(q_pd)
    
    student_ans = q_pd[q_pd.columns[0]]
    print(student_ans)

In [ ]:
def check_keyword(df,a):
    label_encoder = []
    label = 0
    
    for j in student_ans:
      if a in str(j):
          label = 1
      else:
          label = 0
        # print(label)
      label_encoder.append(label)
    print(label_encoder)
    df[a] = label_encoder
    print(q_pd)
    q_pd.to_csv(f"/content/drive/MyDrive/final project/Dataset/output_data/keyword_checker/keyword_checker_{q.split('_')[2].split('.')[0]}.csv")
    return label_encoder


In [ ]:
for i in range(0,n):
  check_keyword(q_pd,keyword_split[i])

In [ ]:
# def check_keyword(df,a):

#   for q in wai_questions:
#     q_pd = pd.read_csv(os.path.join(wai_path,q),index_col=0)
#     print(q_pd.head(3))

#     # 컬럼 명 변경하기(문제 번화 확인을 위해 키워드 컬럼명은 나중에 변경)
#     col_list = list(q_pd.columns)    # 컬럼명 리스트로 변경 후 내용 변경
#     col_list[0] = 'student_ans'
#     q_pd.columns = col_list
#     print(q_pd)

#     # 문항별 키워드 갯수 확인
#     keyword=q_pd.columns[1:]
#     n = len(keyword)
#     print(n)
    
#     # 문항번호와 키워드 분리
#     keyword_split = []
#     for i in range(0,n):
#       split_key= keyword[i].split('_')[1]
#       keyword_split.append(split_key)

#     print(keyword_split)
#     # 컬럼 명 변경하기
#     col_list = list(q_pd.columns)    
#     col_list[1:] = keyword_split          # 분리한 키워드 활용하여 컬럼명 변경/키워드와 동일하게 변경해야 label입력 가능
#     q_pd.columns = col_list
#     print(q_pd)
    
#     student_ans = q_pd[q_pd.columns[0]]
#     print(student_ans)

#     label_encoder = []
#     label = 0
    
#     for j in student_ans:
#       if a in str(j):
#           label = 1
#       else:
#           label = 0
#         # print(label)
#       label_encoder.append(label)
#     print(label_encoder)
#     df[a] = label_encoder
#     print(q_pd)
#     q_pd.to_csv(f"/content/drive/MyDrive/final project/Dataset/output_data/keyword_checker/keyword_checker_{q.split('_')[2].split('.')[0]}.csv")
#     return label_encoder


In [ ]:
# for i in range(0,n):
#   check_keyword(tr_key,keyword_split[i])